In [37]:
import multiprocessing as mp
import numpy as np
import osmnx as ox

np.random.seed(0)
# ox.__version__

In [38]:
place = "Baltimore, Maryland, USA"
G = ox.graph.graph_from_place(place, network_type="drive")
Gp = ox.projection.project_graph(G)

In [39]:
# 在网络几何空间内随机采样 n 个点
# points = ox.utils_geo.sample_points(ox.convert.to_undirected(Gp), n=100)
# X = points.x.values
# Y = points.y.values
# X0 = X.mean()
# Y0 = Y.mean()

In [40]:
# 找到每个点的最近节点，并可选地返回距离
#nodes, dists = ox.distance.nearest_nodes(Gp, X, Y, return_dist=True)


In [41]:
# 找到这些节点之间的最短路径（按距离）并绘制
# orig = list(G)[0]
# dest = list(G)[120]
# route = ox.routing.shortest_path(G, orig, dest, weight="length")
# fig, ax = ox.plot.plot_graph_route(G, route, route_color="g", route_linewidth=3, node_size=0)

In [42]:
# routes = ox.routing.k_shortest_paths(G, orig, dest, k=30, weight="length")
# fig, ax = ox.plot.plot_graph_routes(
#     G, list(routes), route_colors="brown", route_linewidth=1, node_size=0
# )

In [43]:
# 估算所有缺失数据的边缘速度
G = ox.routing.add_edge_speeds(G)

# 计算所有边缘的旅行时间（秒）
G = ox.routing.add_edge_travel_times(G)

In [44]:
# 查看按道路类型分类的平均速度/时间值
edges = ox.convert.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
#edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

In [45]:
# 再次执行相同操作，但这次传递一些默认速度值（公里/小时）
# 以填补 OSM 中缺失 `maxspeed` 的边缘
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.routing.add_edge_speeds(G, hwy_speeds=hwy_speeds)
G = ox.routing.add_edge_travel_times(G)

In [46]:
# 通过最小化旅行距离和旅行时间计算两条路径
# orig = list(G)[1]
# dest = list(G)[120]
# route1 = ox.routing.shortest_path(G, orig, dest, weight="length")
# route2 = ox.routing.shortest_path(G, orig, dest, weight="travel_time")

In [47]:
# 绘制路径
# fig, ax = ox.plot.plot_graph_routes(
#     G, routes=[route1, route2], route_colors=["r", "y"], route_linewidth=6, node_size=0
# )

In [48]:
# # 比较两条路径
# route1_length = int(sum(ox.routing.route_to_gdf(G, route1, weight="length")["length"]))
# route2_length = int(sum(ox.routing.route_to_gdf(G, route2, weight="travel_time")["length"]))
# route1_time = int(sum(ox.routing.route_to_gdf(G, route1, weight="length")["travel_time"]))
# route2_time = int(sum(ox.routing.route_to_gdf(G, route2, weight="travel_time")["travel_time"]))
# print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
# print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")

In [49]:
# 您有多少个 CPU 核心
mp.cpu_count()


20

In [50]:
# 使用随机起点-终点对计算 100,000 条最短路径
# n = 10000
# origs = np.random.choice(G.nodes, size=n, replace=True)
# dests = np.random.choice(G.nodes, size=n, replace=True)
# dests = np.random.choice(G.nodes, size=10, replace=True)
# print(dests.shape)

In [51]:
# # 在网络几何空间内随机采样 n 个点
# points = ox.utils_geo.sample_points(ox.convert.to_undirected(Gp), n=1000)
# X1 = points.x.values
# Y1 = points.y.values

# # 找到每个点的最近节点，并可选地返回距离
# nodes1, dists1 = ox.distance.nearest_nodes(Gp, X1, Y1, return_dist=True)

# origs = nodes1
# dest1 = ox.distance.nearest_nodes(G, X=-76.608411, Y=39.286316)  #国家水族馆分馆
# dest2 = ox.distance.nearest_nodes(G, X=-76.613119, Y=39.280589)  #一个社会组织
# dest3 = ox.distance.nearest_nodes(G, X=-76.602019, Y=39.283081)  #四季酒店 五星级
# dest4 = ox.distance.nearest_nodes(G, X=-76.605554, Y=39.264594)  #roost 酒店 评分最高
# dest5 = ox.distance.nearest_nodes(G, X=-76.581978, Y=39.267149)  #northwest harbor
# dest6 = ox.distance.nearest_nodes(G, X=-76.569818, Y=39.275539)  #百货商场
# dest7 = ox.distance.nearest_nodes(G, X=-76.583514, Y=39.282336)  #杂货店
# dests = [dest1, dest2, dest3, dest4, dest5, dest6, dest7]

In [52]:
#from tqdm.notebook import tqdm  # 进度条显示

# 在网络几何空间内随机采样 n 个点
n = 1000
points = ox.utils_geo.sample_points(ox.convert.to_undirected(Gp), n=n)
X1 = points.x.values
Y1 = points.y.values

# 获取所有起点节点（使用原始图G）
origs, _ = ox.distance.nearest_nodes(G, X1, Y1, return_dist=True)

# 预定义目的地
destinations = [
    ('国家水族馆分馆', -76.608411, 39.286316),
    ('社会组织', -76.613119, 39.280589),
    ('四季酒店', -76.602019, 39.283081),
    ('roost酒店', -76.605554, 39.264594),
    ('northwest harbor', -76.581978, 39.267149),
    ('百货商场', -76.569818, 39.275539),
    ('杂货店', -76.583514, 39.282336)
]

# 获取目的地节点
dest_nodes = {}
for name, x, y in destinations:
    dest_nodes[name] = ox.distance.nearest_nodes(G, X=x, Y=y)

# 计算并存储所有路径
routes_dict = {}

In [53]:
%%time

for dest_name, dest_node in dest_nodes.items():
    dests = [dest_node] * n
    routes = ox.routing.shortest_path(G, origs, dests, weight="travel_time", cpus=18)
    routes_dict[dest_name] = routes

print(f"路径计算完成！已存储 {len(destinations)} 个目的地的路径数据")

路径计算完成！已存储 7 个目的地的路径数据
CPU times: total: 1min 58s
Wall time: 8min 41s


In [59]:
import pandas as pd

results = []
for dest_name, routes in routes_dict.items():
    total_seconds = 0
    valid_count = 0
    for route in routes:
        if route and len(route) > 0:
            try:
                gdf = ox.routing.route_to_gdf(G, route, weight="travel_time")
                time_sum = int(gdf["travel_time"].sum())
                total_seconds += time_sum
                valid_count += 1
            except:
                continue
    
    avg_min = (total_seconds / valid_count / 60) if valid_count > 0 else np.nan
    coverage = valid_count / len(origs)  # 保持为浮点数
    
    results.append({
        'Destination': dest_name,
        'Avg Time (mins)': round(avg_min, 1) if not np.isnan(avg_min) else 'N/A',
        'Coverage': coverage,  # 改为存储原始数值
        'Valid Routes': valid_count
    })

# 创建可视化表格
df = pd.DataFrame(results).set_index('Destination')
# 修正后的格式设置
display(df.style
        .format({
            'Avg Time (mins)': '{:.1f}',  # 格式化浮点数
            'Coverage': '{:.1%}'  # 自动转换数值到百分比
        }, na_rep="N/A")
        .set_caption(f"随机 {n} 个点的可达性分析"))
# %%time
# 使用我计算机上的所有核心求解所有路径需要 2.3 秒
# 我有一台 24 线程的 AMD 5900x：性能取决于您的具体 CPU
# routes = ox.routing.shortest_path(G, origs, dests, weight="travel_time", cpus=None)

,Avg Time (mins),Coverage,Valid Routes
Destination,,,
国家水族馆分馆,14.0,80.8%,808
社会组织,13.9,80.8%,808
四季酒店,14.5,80.8%,808
roost酒店,14.8,80.8%,808
northwest harbor,16.9,80.8%,808
百货商场,15.4,80.8%,808
杂货店,15.2,80.8%,808


In [37]:
#%%time
# 使用我计算机上的 1 个核心求解所有路径需要 29 秒

# routes = ox.routing.shortest_path(G, origs, dests, weight="travel_time", cpus=18)

In [ ]:
# # 我们得到了多少结果
# print(len(routes))

# # 其中有多少是可求解的路径
# # 由于有向图边界效应，有些路径可能无法求解
# routes_valid = [r for r in routes if r is not None]
# print(len(routes_valid))